# Disc Golf Sim by McCann Dahl
Set the inital conditions using the sliders and watch the flight graphs update

In [1]:
from ipywidgets import HBox, VBox, IntSlider, interactive_output, Dropdown, FloatSlider
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits import mplot3d
import frispy
from dataclasses import dataclass

In [2]:
if (frispy.__version__ < '1.1.0'):
    print('You are using frispy version '+frispy.__version__+'. To upgrade your requirements, run `pip install --upgrade --force-reinstall -r requirements.txt`. Also see readme if you have issues with Frispy.')
assert(frispy.__version__ >= '1.1.0')

In [3]:
class MccDisc():
    def __init__(self, disc, name):
        self.disc = disc
        self.name = name
@dataclass
class HummelModel(frispy.Model):
    PL0: float = 0.33
    PLa: float = 1.9
    PD0: float = 0.18
    PTy0: float = -0.082
    PTya: float = 0.43
@dataclass
class AviarModel(frispy.Model):
    PL0: float = 0.152
    PLa: float = 0.044
    PD0: float = 0.083
    PTy0: float = -0.018
    PTya: float = 0.002
@dataclass
class BuzzModel(frispy.Model):
    PL0: float = 0.099
    PLa: float = 0.041
    PD0: float = 0.061
    PTy0: float = -0.033
    PTya: float = 0.004
@dataclass
class RocModel(frispy.Model):
    PL0: float = 0.053
    PLa: float = 0.043
    PD0: float = 0.067
    PTy0: float = -0.015
    PTya: float = 0.003
@dataclass
class FlickModel(frispy.Model):
    PL0: float = 0.100
    PLa: float = 0.038
    PD0: float = 0.076
    PTy0: float = -0.007
    PTya: float = 0.008
@dataclass
class StormModel(frispy.Model):
    PL0: float = 0.107
    PLa: float = 0.045
    PD0: float = 0.057
    PTy0: float = -0.026
    PTya: float = 0.004
@dataclass
class WraithModel(frispy.Model):
    PL0: float = 0.143
    PLa: float = 0.040
    PD0: float = 0.055
    PTy0: float = -0.020
    PTya: float = 0.006
@dataclass
class QuarterModel(frispy.Model):
    PL0: float = 0.138
    PLa: float = 0.039
    PD0: float = 0.065
    PTy0: float = -0.038
    PTya: float = 0.005
all_MccDiscs = [
    MccDisc(frispy.Disc(model=frispy.Model()),'Frispy default'),
    MccDisc(frispy.Disc(model=HummelModel()),'Hummel study'),
    MccDisc(frispy.Disc(model=AviarModel()),'Putter - Aviar'),
    MccDisc(frispy.Disc(model=RocModel()),'Midrange - Roc'),
    MccDisc(frispy.Disc(model=BuzzModel()),'Midrange - Buzz'),
    MccDisc(frispy.Disc(model=WraithModel()),'Driver - Wraith'),
    MccDisc(frispy.Disc(model=FlickModel()),'Driver - Flick'),
    MccDisc(frispy.Disc(model=QuarterModel()),'Driver - Quarter'),
]

In [4]:
number_of_sliders_updated_from_dropdown_change = 0

## Disc Coefficients
Use these sliders to set the disc coefficients. The dropdown will help you select some default disc coefficent sets.

In [5]:
all_coefficients = list(map(lambda x: x.disc.model.PL0, all_MccDiscs))
PL0_slider = FloatSlider(
    min=min(all_coefficients),
    max=max(all_coefficients),
    step=abs(max(all_coefficients)-min(all_coefficients))/20,
    description='PL0:'
)
all_coefficients = list(map(lambda x: x.disc.model.PLa, all_MccDiscs))
PLa_slider = FloatSlider(
    min=min(all_coefficients),
    max=max(all_coefficients),
    step=abs(max(all_coefficients)-min(all_coefficients))/20,
    description='PLa:'
)
all_coefficients = list(map(lambda x: x.disc.model.PD0, all_MccDiscs))
PD0_slider = FloatSlider(
    min=min(all_coefficients),
    max=max(all_coefficients),
    step=abs(max(all_coefficients)-min(all_coefficients))/20,
    description='PD0:'
)
all_coefficients = list(map(lambda x: x.disc.model.PTy0, all_MccDiscs))
PTy0_slider = FloatSlider(
    min=min(all_coefficients),
    max=max(all_coefficients),
    step=abs(max(all_coefficients)-min(all_coefficients))/20,
    description='PTy0:'
)
all_coefficients = list(map(lambda x: x.disc.model.PTya, all_MccDiscs))
PTya_slider = FloatSlider(
    min=min(all_coefficients),
    max=max(all_coefficients),
    step=abs(max(all_coefficients)-min(all_coefficients))/20,
    description='PTya:'
)
all_coefficient_sliders = [PL0_slider,PLa_slider,PD0_slider,PTy0_slider,PTya_slider]

def set_coefficient_sliders(mccdisc):
    if mccdisc is not None:
        PL0_slider.value = mccdisc.disc.model.PL0
        PLa_slider.value = mccdisc.disc.model.PLa
        PD0_slider.value = mccdisc.disc.model.PD0
        PTy0_slider.value = mccdisc.disc.model.PTy0
        PTya_slider.value = mccdisc.disc.model.PTya
selected_index = 4
number_of_sliders_updated_from_dropdown_change = 5
set_coefficient_sliders(all_MccDiscs[selected_index])

In [6]:
discCoefficientsDropdown = Dropdown(
    value=all_MccDiscs[selected_index],
    options=list(map(lambda x: (x.name,x), all_MccDiscs)),
    description='Disc:',
)
def on_change_discCoefficientsDropdown(change):
    global number_of_sliders_updated_from_dropdown_change
    if change['type'] == 'change' and change['name'] == 'value':
        newDiscSelected = change['new']
        number_of_sliders_updated_from_dropdown_change = 5
        set_coefficient_sliders(newDiscSelected)

discCoefficientsDropdown.observe(on_change_discCoefficientsDropdown)

In [7]:
display(HBox([discCoefficientsDropdown, VBox(all_coefficient_sliders)]))

## Disc throw
Use these sliders to determine the x y and z rotations and speeds of the disc as it leaves the thrower's hand

In [8]:
x_slider = FloatSlider(
    min=-5,
    max=5,
    step=0.1,
    description='x (m):'
)
y_slider = FloatSlider(
    min=0,
    max=5,
    step=0.1,
    description='y (m):'
)
z_slider = FloatSlider(
    value=1,
    min=0,
    max=5,
    step=0.1,
    description='z (m):'
)
vx_slider = FloatSlider(
    value=30,
    min=0,
    max=200,
    step=0.1,
    description='x vel (m/s):'
)
vy_slider = FloatSlider(
    min=-40,
    max=40,
    step=0.1,
    description='y vel (m/s):'
)
vz_slider = FloatSlider(
    min=-40,
    max=40,
    step=0.1,
    description='z vel (m/s):'
)
phi_slider = FloatSlider(
    value=0.36,
    min=-1,
    max=1,
    step=0.01,
    description='phi (rad):'
)
theta_slider = FloatSlider(
    min=-1,
    max=1,
    step=0.01,
    description='theta (rad):'
)
gamma_slider = FloatSlider(
    min=-3.14,
    max=3.14,
    step=0.1,
    description='gamma (rad):'
)
dphi_slider = FloatSlider(
    min=-1,
    max=1,
    step=0.01,
    description='dphi (rad/s):'
)
dtheta_slider = FloatSlider(
    min=-1,
    max=1,
    step=0.01,
    description='dtheta (rad/s):'
)
dgamma_slider = FloatSlider(
    value=60,
    min=-400,
    max=400,
    step=0.1,
    description='dgamma (rad/s):'
)
all_throw_sliders = [x_slider,y_slider,z_slider,vx_slider,vy_slider,vz_slider,
phi_slider,theta_slider,gamma_slider,dphi_slider,dtheta_slider,dgamma_slider]

In [9]:
discThrowDropdown = Dropdown(
    options=[
        ('Backhand',1),
        ('Fronthand',-1),
    ],
    description='Type:',
)
def set_discThrowDropdown_sliders(index):
    global number_of_sliders_updated_from_dropdown_change
    number_of_sliders_updated_from_dropdown_change = 1
    dgamma_slider.value = 60 * index
def on_change_discThrowDropdown(change):
    if change['type'] == 'change' and change['name'] == 'value':
        index = change['new']
        set_discThrowDropdown_sliders(index)

discThrowDropdown.observe(on_change_discThrowDropdown)

In [10]:
discAngleDropdown = Dropdown(
    options=[
        ('Anhyzer',1),
        ('Hyzer',-1),
    ],
    description='Angle:',
)
def set_discAngleDropdown_sliders(index):
    global number_of_sliders_updated_from_dropdown_change
    number_of_sliders_updated_from_dropdown_change = 1
    phi_slider.value = 0.36 * index
def on_change_discAngleDropdown(change):
    if change['type'] == 'change' and change['name'] == 'value':
        index = change['new']
        set_discAngleDropdown_sliders(index)

discAngleDropdown.observe(on_change_discAngleDropdown)

In [11]:
display(HBox([VBox([discThrowDropdown,discAngleDropdown]), VBox(all_throw_sliders[0:6]), VBox(all_throw_sliders[6:12])]))

## Graphs
Here is the simulated throw of the above configuration

In [12]:
# taken from stack overflow
def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([0, (z_middle + plot_radius)*2])

In [13]:
def get_ax(fig,x,y,z,phi,subplotnum, title,azim=0,elev=0):
    radius = 0.2
    X = x
    Y = y
    Z = z
    X1 = X
    X2 = X
    Y1 = Y + radius * np.cos(phi)
    Y2 = Y - radius * np.cos(phi)
    Z1 = Z + radius * np.sin(phi)
    Z2 = Z - radius * np.sin(phi)
    
    ax = fig.add_subplot(subplotnum, projection='3d')
    ax.set_proj_type('ortho')
    ax.plot3D(X, Y, Z, 'red')
    ax.plot3D(X1, Y1, Z1, 'red')
    ax.plot3D(X2, Y2, Z2, 'red')
    ax.view_init(elev, azim)
    set_axes_equal(ax)
    ax.set_box_aspect([1,1,1])
    ax.set_title(title)
    

def interactive_refresh_plot(x,y,z,phi):
    fig = plt.figure(figsize = [14, 14])
    
    ax1 = get_ax(fig,x,y,z,phi,221,'3d view', 135,25)
    ax2 = get_ax(fig,x,y,z,phi,222, 'side view', -90)
    ax3 = get_ax(fig,x,y,z,phi,223, 'top view', 180, 90)
    ax4 = get_ax(fig,x,y,z,phi,224, 'thrower\'s view', 180)
    
    plt.show()

In [14]:
def compute_custom_trajectory(
        PL0,
        PLa,
        PD0,
        PTy0,
        PTya,
        x,
        y,
        z,
        vx,
        vy,
        vz,
        phi,
        theta,
        gamma,
        dphi,
        dtheta,
        dgamma,
        alpha_0 = -4 * np.pi / 180, # THIS IS AN ISSUE WITH FRISPY. THIS SHOULD BE -4 not 4
        PDa = None,
        PTxwx = None,
        PTxwz = None,
        PTywy = None,
        PTzwz = None,
        area = None,  # m^2
        I_zz = None,  # kg*m^2
        I_xx = None,  # kg*m^2
        mass = None,  # kg
    ):
    disc = frispy.Disc()
    disc.model.PL0 = PL0
    disc.model.PLa = PLa
    disc.model.PD0 = PD0
    disc.model.PTy0 = PTy0
    disc.model.PTya = PTya
    disc.model.alpha_0 = alpha_0
    if PDa is not None:
        disc.model.PDa = PDa
    if PTxwx is not None:
        disc.model.PTxwx = PTxwx
    if PTxwz is not None:
        disc.model.PTxwz = PTxwz
    if PTywy is not None:
        disc.model.PTywy = PTywy
    if PTzwz is not None:
        disc.model.PTzwz = PTzwz
    if area is not None:
        disc._default_physical_attributes.area = area
    if I_zz is not None:
        disc._default_physical_attributes.I_zz = I_zz
    if I_xx is not None:
        disc._default_physical_attributes.I_xx = I_xx
    if mass is not None:
        disc._default_physical_attributes.mass = mass
    disc.default_initial_conditions['x'] = x
    disc.default_initial_conditions['y'] = y
    disc.default_initial_conditions['z'] = z
    disc.default_initial_conditions['vx'] = vx
    disc.default_initial_conditions['vy'] = vy
    disc.default_initial_conditions['vz'] = vz
    disc.default_initial_conditions['phi'] = phi
    disc.default_initial_conditions['theta'] = theta
    disc.default_initial_conditions['gamma'] = gamma
    disc.default_initial_conditions['dphi'] = dphi
    disc.default_initial_conditions['dtheta'] = dtheta
    disc.default_initial_conditions['dgamma'] = dgamma
    disc.reset_initial_conditions()
    result = disc.compute_trajectory(flight_time=10,n_times=1000)
    return result

In [15]:
def plot_func(
        PL0,
        PLa,
        PD0,
        PTy0,
        PTya,
        x,
        y,
        z,
        vx,
        vy,
        vz,
        phi,
        theta,
        gamma,
        dphi,
        dtheta,
        dgamma
    ):
    result = compute_custom_trajectory(
        PL0,
        PLa,
        PD0,
        PTy0,
        PTya,
        x,
        y,
        z,
        vx,
        vy,
        vz,
        phi,
        theta,
        gamma,
        dphi,
        dtheta,
        dgamma
    )
    times = result.times
    x, y, z, phi = result.x, result.y, result.z, result.phi
    interactive_refresh_plot(x,y,z, phi)

In [16]:
out = interactive_output(plot_func, {
    "PL0": PL0_slider,
    "PLa": PLa_slider,
    "PD0": PD0_slider,
    "PTy0": PTy0_slider,
    "PTya": PTya_slider,
    "x": x_slider,
    "y": y_slider,
    "z": z_slider,
    "vx": vx_slider,
    "vy": vy_slider,
    "vz": vz_slider,
    "phi": phi_slider,
    "theta": theta_slider,
    "gamma": gamma_slider,
    "dphi": dphi_slider,
    "dtheta": dtheta_slider,
    "dgamma": dgamma_slider,
})

In [17]:
display(out)

Output()

All distances are given in meters. These graphs may look like they have unnecessary empty space - this is to ensure that the perspectives and visualizations are to-scale and accurate. 
If you have any questions or comments about this simulation, contact McCann Dahl mccdahl@gmail.com

In [18]:
### clear dropdown when slider updated manually
def clear_dropdown_discCoefficientsDropdown(PL0,PLa,PD0,PTy0,PTya):
    global number_of_sliders_updated_from_dropdown_change
    if number_of_sliders_updated_from_dropdown_change > 0:
        number_of_sliders_updated_from_dropdown_change -= 1
    else:
        discCoefficientsDropdown.value = None
        
interactive_output(clear_dropdown_discCoefficientsDropdown, {
    "PL0": PL0_slider,
    "PLa": PLa_slider,
    "PD0": PD0_slider,
    "PTy0": PTy0_slider,
    "PTya": PTya_slider,
})

def clear_dropdown_discThrowDropdown(dgamma_slider_value):
    global number_of_sliders_updated_from_dropdown_change
    if number_of_sliders_updated_from_dropdown_change > 0:
        number_of_sliders_updated_from_dropdown_change -= 1
    else:
        discThrowDropdown.value = None
        
interactive_output(clear_dropdown_discThrowDropdown, {
    "dgamma_slider_value": dgamma_slider,
})

def clear_dropdown_discAngleDropdown(phi_slider_value):
    global number_of_sliders_updated_from_dropdown_change
    if number_of_sliders_updated_from_dropdown_change > 0:
        number_of_sliders_updated_from_dropdown_change -= 1
    else:
        discAngleDropdown.value = None
        
interactive_output(clear_dropdown_discAngleDropdown, {
    "phi_slider_value": phi_slider,
})

Output()